# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Подгружаем необходимые бибилиотеки.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.utils import shuffle

Считываем файл. Изучаем его характеристики. Видно, что столбец 'RowNumber' просто дублирует номер строки, его можно удалить. Столбец 'CustomerId' служит идентификатором. Проверяем его на дубликаты, убеждаемся, что их нет и удаляем его. Также убираем столбец с фамилиями. Видно, что в столбце 'Tenure' есть пустые значения. Столбец хранит данные о том, сколько лет пользователь является клиентом. Заполним пропуски заглушками (-1), чтобы не потерять эти данные, так как их около 10% от всех данных. Также видно, что в столбце с названием страны может быть всего 3 разные категории.

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
display(data.head()) 
data.info()
data.shape
data = data.drop('RowNumber', axis=1)
data['CustomerId'].value_counts().sum()
data = data.drop('CustomerId', axis=1)
data = data.drop('Surname', axis=1)

print(data['Geography'].unique())

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
['France' 'Spain' 'Germany']


<span style="color:blue">Заполним пустные значения в столбце Tenure нулями, предположив, что это новые клиенты.</span>

In [3]:
data['Tenure'] = data['Tenure'].fillna(0)

Проверяем обновленную таблицу. Новый размер 10000 на 11

In [4]:
display(data.head())
data.shape

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


(10000, 11)

Далее подготовим данные методом OHE, что модель могла правильно использовать категориальные данные. Он позволит использовать модели разных типов. Используем параметр drop_first=True, чтобы не попасть в дамми-ловушку. Тем самым, столбец с названием стран даст 2 столбца вместо 1, а Гендер оставит 1. Таким образом, количество столбцов увеличится на 1.

In [5]:
data = pd.get_dummies(data, drop_first=True)
data.shape

(10000, 12)

Теперь разделим данные на тренировочную, валидационную и тренировочную выборки, в пропориии 3:1:1 и проверим размеры новых выборок. 

In [6]:
features = data.drop('Exited', axis=1)
target = data['Exited']
features_train, features_x, target_train, target_x = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_x, target_x, test_size=0.5, random_state=12345)
print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)


(6000, 11) (2000, 11) (2000, 11)
(6000,) (2000,) (2000,)


0       1
1       0
2       1
3       0
4       0
       ..
9995    0
9996    0
9997    1
9998    1
9999    0
Name: Exited, Length: 10000, dtype: int64

Выполним масштабирование признаков во всех выборках, сперва выделив количественные признаки.

In [7]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
display(features_train.head())
features_valid[numeric] = scaler.transform(features_valid[numeric])
display(features_valid.head())
features_test[numeric] = scaler.transform(features_test[numeric])
display(features_test.head())


/tmp/ipykernel_152/1289266845.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric] = scaler.transform(features_train[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7479,-0.886751,-0.373192,1.104696,1.232271,-0.891560,1,0,-0.187705,0,1,1
3411,0.608663,-0.183385,1.104696,0.600563,-0.891560,0,0,-0.333945,0,0,0
6027,2.052152,0.480939,-0.503694,1.027098,0.830152,0,1,1.503095,1,0,1
1247,-1.457915,-1.417129,0.461340,-1.233163,0.830152,1,0,-1.071061,0,0,1
3716,0.130961,-1.132419,-0.825373,1.140475,-0.891560,0,0,1.524268,1,0,0


/tmp/ipykernel_152/1289266845.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_valid[numeric] = scaler.transform(features_valid[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
8532,-0.699824,-0.373192,-0.825373,-1.233163,0.830152,1,0,-0.015173,0,0,0
5799,-0.284431,0.575842,-0.503694,-1.233163,-0.891560,1,1,1.471724,0,0,0
5511,0.151731,-0.657902,-1.468729,0.438711,-0.891560,1,0,-1.367107,1,0,1
7365,-0.876366,-0.278288,1.748053,1.239884,-0.891560,1,1,-0.786517,0,1,0
7367,-0.481743,0.291132,1.748053,-1.233163,0.830152,1,0,1.358533,0,1,1


/tmp/ipykernel_152/1289266845.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_test[numeric] = scaler.transform(features_test[numeric])
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7041,-2.226392,-0.088482,-0.825373,-1.233163,0.830152,1,0,0.647083,0,0,1
5709,-0.087120,0.006422,1.426375,-1.233163,-0.891560,1,0,-1.658410,0,0,0
7117,-0.917905,-0.752805,0.139662,0.722307,-0.891560,1,1,-1.369334,0,1,1
7775,-0.253277,0.101325,1.748053,-1.233163,0.830152,1,0,0.075086,0,1,1
8735,0.785204,-0.847708,1.748053,0.615625,-0.891560,0,1,-1.070919,0,0,1


## Исследование задачи

Обучим 3 разные модели и посчитаем для них Accuracy. Для дерева решений получилось 0.858, для случайного леса 0.864, для регрессии 0.8

In [8]:
best_result_1 = 0
for depth in range(1, 20):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_1.fit(features_train,target_train)  
    result = model_1.score(features_valid, target_valid)
    if result > best_result_1:
        tree_model = model_1
        best_result_1 = result
        
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result_1, tree_model)

best_result_2 = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_2.fit(features_train, target_train) 
        result = model_2.score(features_valid, target_valid) 
        if result > best_result_2:
            forest_model = model_2
            best_result_2 = result

print("Accuracy наилучшей модели леса на валидационной выборке:", best_result_2, forest_model)

reg_model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
reg_model.fit(features_train, target_train) 
best_result_3 = reg_model.score(features_valid, target_valid)
print("Accuracy регрессии на валидационной выборке:", best_result_3)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.858 DecisionTreeClassifier(max_depth=6, random_state=12345)
Accuracy наилучшей модели леса на валидационной выборке: 0.864 RandomForestClassifier(max_depth=8, n_estimators=40, random_state=12345)
Accuracy регрессии на валидационной выборке: 0.8025


Рассмотрим другие метрики.

In [9]:
print("Полнота дерево" , recall_score(target_valid, tree_model.predict(features_valid)))
print("Точность дерево", precision_score(target_valid, tree_model.predict(features_valid)))
print("F1-мера дерево", f1_score(target_valid, tree_model.predict(features_valid)))
print("AUC-ROC дерево", roc_auc_score(target_valid, tree_model.predict(features_valid)))
print('')
print("Полнота лес" , recall_score(target_valid, forest_model.predict(features_valid)))
print("Точность лес", precision_score(target_valid, forest_model.predict(features_valid)))
print("F1-мера лес", f1_score(target_valid, forest_model.predict(features_valid)))
print("AUC-ROC лес", roc_auc_score(target_valid, forest_model.predict(features_valid)))
print('')
print("Полнота регрессия" , recall_score(target_valid, reg_model.predict(features_valid)))
print("Точность регрессия", precision_score(target_valid, reg_model.predict(features_valid)))
print("F1-мера регрессия", f1_score(target_valid, reg_model.predict(features_valid)))
print("AUC-ROC регрессия", roc_auc_score(target_valid, reg_model.predict(features_valid)))

Полнота дерево 0.44976076555023925
Точность дерево 0.7768595041322314
F1-мера дерево 0.5696969696969697
AUC-ROC дерево 0.7078133789824521

Полнота лес 0.4258373205741627
Точность лес 0.8476190476190476
F1-мера лес 0.5668789808917197
AUC-ROC лес 0.702804880261797

Полнота регрессия 0.23684210526315788
Точность регрессия 0.5657142857142857
F1-мера регрессия 0.33389544688026984
AUC-ROC регрессия 0.5944008250715284


Показатели довольно низкие, f1-мера не удовлетворяет поставленной задаче (>0.59). Полнота ниже 0.5, что говорит о том, что модель предскажет меньше половины от реальных уходов. Это неудовлетворительный результат.

## Борьба с дисбалансом

Проверим баланс классов.

In [10]:
print(target_train.value_counts())
print(target_valid.value_counts())
print(target.value_counts())

0    4804
1    1196
Name: Exited, dtype: int64
0    1582
1     418
Name: Exited, dtype: int64
0    7963
1    2037
Name: Exited, dtype: int64


<div style="border:solid purple 5px; padding: 20px"> 
<h2 align="center"> Рубрика «Питонячий лайфхакер» <a class="tocSkip"> </h2>

<h3> Индикаторы состояния: progress и tqdm <a class="tocSkip"> </h3>

Как перестать беспокоиться о том, не завис ли скрипт? Вы можете создавать свои индикаторы состояния. Это весьма увлекательное занятие. Но делать то же самое с [progress](https://pypi.org/project/progress/) (ее обычно используют для работы из консоли) или [tqdm](https://pypi.org/project/tqdm/) (отлично подойдет для тетрадок) куда быстрее и надежнее.

Обычно я использую эти две библиотеки, но вот [здесь](https://habr.com/ru/post/483400/) ты можешь почитать про альтернативные варианты

![](https://i.ibb.co/B4SDm8Y/68747470733a2f2f7261772e6769746875622e636f6d2f7665726967616b2f70726f67726573732f6d61737465722f64656d.gif)

Баланс классов стремится к 0 у обеих выборок примерно в соотношении 1:4. Модель будет склоняться к 0. Проверим это на разных моделях. Также проверим модели на адекватность константной моделью.

In [11]:
print(pd.Series(tree_model.predict(features_valid)).value_counts())
print(pd.Series(forest_model.predict(features_valid)).value_counts())
print(pd.Series(reg_model.predict(features_valid)).value_counts())

target_constant = pd.Series([0]*len(target_valid))
print(accuracy_score(target_valid, target_constant))

0    1758
1     242
dtype: int64
0    1790
1     210
dtype: int64
0    1825
1     175
dtype: int64
0.791


Предположение подтвердилось, при этом "перекос" у модели регрессии практически 1:10. При констатной модели точность 0.791, что очень близко к точности модели регрессии. Очевидно, необходимо решить задачу дисбаланса классов. Для этого восвольщуемся техникой upsampling.
Преобразование проходит в несколько этапов:
1. Разделить обучающую выборку на объекты по классам;
2. Определить тот класс, который содержит меньше объектов. Назовём его меньшим классом;
3. Скопировать несколько раз объекты меньшего класса;
4. С учётом полученных данных создать новую обучающую выборку;
5. Перемешать данные.
Выборку с положительными ответами нужно умножить в 4 раза.
Видим, что количество положительных и отрицательных исходов почти сравнялось.

In [12]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4)
print(target_train_upsampled.value_counts())




0    4804
1    4784
Name: Exited, dtype: int64


Теперь обучим модели на увеличенной выборке. и посчитаем Accuracy.

In [13]:
best_result_1 = 0
for depth in range(1, 20):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_1.fit(features_train_upsampled,target_train_upsampled)  
    result = model_1.score(features_valid, target_valid)
    if result > best_result_1:
        tree_model = model_1
        best_result_1 = result
        
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result_1, tree_model)

best_result_2 = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_2.fit(features_train_upsampled, target_train_upsampled) 
        result = model_2.score(features_valid, target_valid) 
        if result > best_result_2:
            forest_model = model_2
            best_result_2 = result

print("Accuracy наилучшей модели леса на валидационной выборке:", best_result_2, forest_model)

reg_model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
reg_model.fit(features_train_upsampled, target_train_upsampled) 
best_result_3 = reg_model.score(features_valid, target_valid)
print("Accuracy регрессии на валидационной выборке:", best_result_3)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.8105 DecisionTreeClassifier(max_depth=5, random_state=12345)
Accuracy наилучшей модели леса на валидационной выборке: 0.824 RandomForestClassifier(max_depth=10, n_estimators=20, random_state=12345)
Accuracy регрессии на валидационной выборке: 0.701


<div class="alert alert-success">


Кстати, можно было уйти от вложенных циклов и генерить комбинации гиперпараметров с помощью библиотеки `itertools`. Ниже небольшой пример
    
![](https://i.ibb.co/BnJfPMk/image.png)

Для дерева решений получилось 0.81, для случайного леса 0.8245, для регрессии 0.7. Показатели стали только хуже. Но теперь стоит изучить изменение остальных метрик.

In [14]:
print("Полнота дерево" , recall_score(target_valid, tree_model.predict(features_valid)))
print("Точность дерево", precision_score(target_valid, tree_model.predict(features_valid)))
print("F1-мера дерево", f1_score(target_valid, tree_model.predict(features_valid)))
print("AUC-ROC дерево", roc_auc_score(target_valid, tree_model.predict(features_valid)))
print('')
print("Полнота лес" , recall_score(target_valid, forest_model.predict(features_valid)))
print("Точность лес", precision_score(target_valid, forest_model.predict(features_valid)))
print("F1-мера лес", f1_score(target_valid, forest_model.predict(features_valid)))
print("AUC-ROC лес", roc_auc_score(target_valid, forest_model.predict(features_valid)))
print('')
print("Полнота регрессия" , recall_score(target_valid, reg_model.predict(features_valid)))
print("Точность регрессия", precision_score(target_valid, reg_model.predict(features_valid)))
print("F1-мера регрессия", f1_score(target_valid, reg_model.predict(features_valid)))
print("AUC-ROC регрессия", roc_auc_score(target_valid, reg_model.predict(features_valid)))

#Данные без балансировки
#Полнота дерево 0.44976076555023925
#Точность дерево 0.7768595041322314
#F1-мера дерево 0.5696969696969697
#AUC-ROC дерево 0.7078133789824521
#Полнота лес 0.43779904306220097
#Точность лес 0.8356164383561644
#F1-мера лес 0.5745682888540031
#AUC-ROC лес 0.7075215190026555
#Полнота регрессия 0.23684210526315788
#Точность регрессия 0.5657142857142857
#F1-мера регрессия 0.33389544688026984
#AUC-ROC регрессия 0.5944008250715284

Полнота дерево 0.6698564593301436
Точность дерево 0.5374280230326296
F1-мера дерево 0.5963791267305644
AUC-ROC дерево 0.7587588238496482

Полнота лес 0.6650717703349283
Точность лес 0.5673469387755102
F1-мера лес 0.6123348017621146
AUC-ROC лес 0.7655320924999546

Полнота регрессия 0.6842105263157895
Точность регрессия 0.3803191489361702
F1-мера регрессия 0.4888888888888888
AUC-ROC регрессия 0.6948233415396898


Очевидно, что несмотря на снизившийся показатель accuracy, почти все остальные метрики улучшились. Лучший показатель f1 у случайного леса, около 0.62, при требуемых по заданию не менее 0.59. Таким образом, поставленная задача выполняется.

<span style="color:blue">Теперь попробуем решить проблему дисбаланса классов техникой downsampling.
Преобразование проходит в несколько этапов:
1. Разделить обучающую выборку на объекты по классам;
2. Определить тот класс, который содержит больше объектов. Назовём его большим классом;
3. Случайным образом отбросить часть из объектов большего класса;
4. С учётом полученных данных создать новую обучающую выборку;
5. Перемешать данные.</span>

In [15]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled
features_train_downsampled, target_train_downsampled = downsample(features_train, target_train, 0.25)
print(target_train_downsampled.value_counts())


0    1201
1    1196
Name: Exited, dtype: int64


Получили около 1200 строк в обоих классах.

In [16]:
best_result_1 = 0
for depth in range(1, 20):
    model_1 = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_1.fit(features_train_downsampled,target_train_downsampled)  
    result = model_1.score(features_valid, target_valid)
    if result > best_result_1:
        tree_model_down = model_1
        best_result_1 = result
        
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result_1, tree_model)

best_result_2 = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_2 = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_2.fit(features_train_downsampled, target_train_downsampled) 
        result = model_2.score(features_valid, target_valid) 
        if result > best_result_2:
            forest_model_down = model_2
            best_result_2 = result

print("Accuracy наилучшей модели леса на валидационной выборке:", best_result_2, forest_model)

reg_model_down = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000)
reg_model_down.fit(features_train_downsampled, target_train_downsampled) 
best_result_3 = reg_model_down.score(features_valid, target_valid)
print("Accuracy регрессии на валидационной выборке:", best_result_3)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.799 DecisionTreeClassifier(max_depth=5, random_state=12345)
Accuracy наилучшей модели леса на валидационной выборке: 0.803 RandomForestClassifier(max_depth=10, n_estimators=20, random_state=12345)
Accuracy регрессии на валидационной выборке: 0.7025


Точность во всех моделях, кроме регресиии снизилась. В модели регрессии точность не изменилась. Проверим другие метрики. И сравним с метриками, полученными с помощью upsample.

Предыдущие значения:
Полнота лес 0.6650717703349283
Точность лес 0.5673469387755102
F1-мера лес 0.6123348017621146
AUC-ROC лес 0.7655320924999546

In [17]:
print("Полнота" , recall_score(target_test, forest_model_down.predict(features_test)))
print("Точность", precision_score(target_test, forest_model_down.predict(features_test)))
print("F1-мера", f1_score(target_test, forest_model_down.predict(features_test)))
print("AUC-ROC", roc_auc_score(target_test, forest_model_down.predict(features_test)))

Полнота 0.6737588652482269
Точность 0.504424778761062
F1-мера 0.576923076923077
AUC-ROC 0.7481032753634921


Значительно ухудшилась точность, а также f1-мера теперь не входит в удовлетворящие условию значения. Очевидно,метод upsmpling удачнее. Это ожидаемо, учитывая не очень большое количество строк в исходной таблице. Данный метод лучше использовать при очень большом количестве строк.

## Тестирование модели

Проверим модель на тестовой выборке.

In [18]:
print("Полнота" , recall_score(target_test, forest_model.predict(features_test)))
print("Точность", precision_score(target_test, forest_model.predict(features_test)))
print("F1-мера", f1_score(target_test, forest_model.predict(features_test)))
print("AUC-ROC", roc_auc_score(target_test, forest_model.predict(features_test)))

Полнота 0.6524822695035462
Точность 0.5411764705882353
F1-мера 0.5916398713826367
AUC-ROC 0.7520496318982537


F1 на тестовой выборке 0.59, что удовлетворяет поставленной задаче.

### Вывод

1. Мы обработали первоначальные данные, удалив лишнюю информаию и обработав категориальные переменные методом OHE, а количественные масштабированием. 

2. Мы обучили и протестировали модели на получившейся выборке без учета дизбаланса.

3. Мы увидели  дисбаланс классов в пропорции 1:4 в пользу отрицательных и устранили его методом upsampling, увеличив количество элеиентов положитедьного класса в 4 раза. Также попробовали устранить его методом downsampling, но это не принесло желаемых результатов.

4. С использованием обновленных данных, метрики исследования улучшились и достигли требуемых заданием величин. Лучшей моделью оказался алгоритм случайного леса, с показателями Полнота 0.6722488038277512, Точность 0.5676767676767677, F1-мера 0.615553121577218, AUC-ROC 0.7684884979947859 при параметрах max_depth=8, n_estimators=40.

5. Результаты работы модели с тестовой выборкой так же удовлетворили условию. А именно: Полнота 0.6643026004728132 Точность 0.5676767676767677 F1-мера 0.6122004357298474 AUC-ROC 0.7643009514729316

Значение метрики f1 говорит о том, что модель показывает адекватные результаты. Достаточно высокий показатель полноты говорит о том, что модель достаточно хорошо предсказывает уход клиента, если он произойдет. При этом показатель точности ниже, хоть и выше 0.5. Это означает, что ошибочных предсказаний об уходе будет достаточно много. При этом показатель AUC-ROC значительно выше 0.5, что говорит о том, что модель намного качественнее случайной. Вероятно, более точное предсказание реального ухода клиента важнее, чем ошибочное предсказание о его уходе. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [х]  Весь код выполняется без ошибок
- [х]  Ячейки с кодом расположены в порядке исполнения
- [х]  Выполнен шаг 1: данные подготовлены
- [х]  Выполнен шаг 2: задача исследована
    - [х]  Исследован баланс классов
    - [х]  Изучены модели без учёта дисбаланса
    - [х]  Написаны выводы по результатам исследования
- [х]  Выполнен шаг 3: учтён дисбаланс
    - [х]  Применено несколько способов борьбы с дисбалансом
    - [х]  Написаны выводы по результатам исследования
- [х]  Выполнен шаг 4: проведено тестирование
- [х]  Удалось достичь *F1*-меры не менее 0.59
- [х]  Исследована метрика *AUC-ROC*